In [ ]:
from gurobipy import *  #引用gurobi求解包
import numpy as np


def Pattern_Appro_solver(s,timebound):
#Model
    OP = Model("PA")
#Number of variables
    P = len(s.Pi)
    J = int(s.number_of_batch)
    B = len(s.mb)
    numberperunit = int(s.number_of_order / s.number_of_batch)
#Add Vars
    y = OP.addVars(P, J, vtype=GRB.INTEGER, name="y") 
    delta = OP.addVars(J, B, vtype=GRB.BINARY, name="delta")
    for j in range(J):
        for p in range(P):
            if s.Qp[p] == 1:
                y[p, j].vtype = GRB.BINARY
            else:
                y[p, j].ub = s.Qp[p]
#InitialIndex
    for i in s.InitialIndex:
        for j, k, l in s.Pi:
            if i in k and len(k) == 1:
                y[j, s.InitialIndex.index(i)].lb = 1
                for t in range(s.InitialIndex.index(i)) or range(
                        s.InitialIndex.index(i) + 1, J):
                    y[j, t].ub = 0
                break
            elif i in k:
                y[j, s.InitialIndex.index(i)].lb = 1
                y[j, s.InitialIndex.index(i)].ub = 1
                break


#Set objective
    OP.setObjective(quicksum(delta[j, b] for j in range(J) for b in range(B)),
                    GRB.MINIMIZE)
#Add Constr
    OP.addConstrs(sum(y[p, j] for p in range(P)) == numberperunit for j in range(J))
    OP.addConstrs(sum(y[p, j] for j in range(J)) == s.Qp[p] for p in range(P))
    OP.addConstrs(numberperunit * delta[j, b] >= sum(y[p, j] for p in s.Pb[b])
                  for j in range(J) for b in range(B))
#Set time and Log file
    OP.Params.timelimit = timebound
    OP.Params.LogToConsole = 0
    FileStr = './Log/' + 'Pattern_Appro_solver' + '_' + str(s.number_of_order) + '_' + 'order' + '_' + str(s.number_of_batch) + '_' + 'batch' + '.log'
    OP.Params.LogFile = FileStr
    
    OP.update()
    OP.optimize()

#Save solution
    Pi=copy.deepcopy(s.Pi)
    #Pi=s.Pi
    OP.setParam("SolutionNumber", 0)
    vp = OP.getAttr('xn', y)
    vy = np.zeros((s.number_of_order, J))
    for p in range(P):
        for j in range(J):
            if vp[p, j] > 0:
                for k in range(round(vp[p, j])):
                    vy[Pi[p][1][-1]][j] = 1
                    Pi[p][1].pop()
    matrix = [[] for j in range(J)]
    for j in range(J):
        matrix[j].extend(list(np.where(vy[:, j] == 1)[0]))
    s.Pattern_Appro_matrix = matrix
    return OP.ObjVal,OP.MIPGap